In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# Замініть X_train, y_train на ваші дані для навчання моделі
import pandas as pd 
df = pd.read_csv("Wine.csv")
#df.drop('Unnamed: 0', axis=1, inplace=True)
df

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


In [14]:
X = df.drop('Customer_Segment', axis=1)  # X - ознаки, без цільової змінної
y = df['Customer_Segment']  # y - цільова змінна

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
sc = StandardScaler()
 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
# Конвейєр 1
pipeline1 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('model', SVC())  # Змініть модель на іншу SVM, яку ви хочете використовувати
])

param_grid1 = {
    'pca__n_components': [2, 5, 10],
    'model__C': [0.1, 1, 10],
    'model__kernel': ['linear', 'rbf']
}

grid1 = GridSearchCV(pipeline1, param_grid1, cv=3)
grid1.fit(X_train, y_train)

# Конвейєр 2
pipeline2 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', KNeighborsClassifier())  # Змініть модель на іншу KNN, яку ви хочете використовувати
])

param_grid2 = {
    'model__n_neighbors': [3, 5, 7],
    'model__weights': ['uniform', 'distance']
}

grid2 = GridSearchCV(pipeline2, param_grid2, cv=3)
grid2.fit(X_train, y_train)

# Конвейєр 3
pipeline3 = Pipeline([
    ('scaler1', MinMaxScaler()),
    ('scaler2', StandardScaler()),
    ('model', DecisionTreeClassifier())  # Змініть модель на іншу Decision Tree, яку ви хочете використовувати
])

param_grid3 = {
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10]
}

grid3 = GridSearchCV(pipeline3, param_grid3, cv=3)
grid3.fit(X_train, y_train)

# Конвейєр 4
pipeline4 = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('model', RandomForestClassifier())  # Змініть модель на іншу Random Forest, яку ви хочете використовувати
])

param_grid4 = {
    'pca__n_components': [2, 5, 10],
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20]
}

grid4 = GridSearchCV(pipeline4, param_grid4, cv=3)
grid4.fit(X_train, y_train)

# Оцінка результатів та вибір найкращого конвейєра
models = {
    'SVM': grid1,
    'KNN': grid2,
    'DecisionTree': grid3,
    'RandomForest': grid4
}

best_model_name = max(models, key=lambda k: models[k].best_score_)
best_model = models[best_model_name]

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Найкраща модель: {best_model_name}")
print(f"Точність на тестовому датасеті: {accuracy}")

Найкраща модель: SVM
Точність на тестовому датасеті: 1.0
